# Ctree

## Set working directory
Warning: only run the cell below once per kernel session
Warning: the cell below set the working directory to the project root and should therefore be run once per kernel session.

In [ ]:
import os
from pathlib import Path

project_root = Path.cwd().parent
os.chdir(project_root)
print(os.getcwd())

## Load the data

In [ ]:
from src.data import Data
import tomllib

with open("config.toml", "rb") as file:
    config = tomllib.load(file)
    
data = Data(config)

# new mixing matrix has different sample separator
data.replace_sample_sep(".")

In [ ]:
# fetch the current state of the mixing matrix with a tumor types column
dataset = data.get_mm_with_tt()

## Split data into train, test, and validation sets

In [ ]:
# Set the sizes of the train, test, and validation sets according to some fraction
train, test, val = data.get_train_test_val(
    train_size=0.7,
    test_size=0.15,
    val_size=0.15,
    data=dataset
)

## Build the ctree model

The plot, save, and assess methods will save files to the output directory specified in the config.toml file.
Each model type (e.g., glm, ctree, cforest) will have its own subdirectory in the output directory.
Each model fit will have its own subdirectory in the model type directory consisting of the date, time, and a unique identifier.

In [ ]:
from src.ml.ctree import Ctree
ctree = Ctree(data)

ctrl = ctree.CtreeControl(
    testtype= "Bonferroni",
    alpha=0.01
)

ctree.fit(
    train=train,
    ctree_control=ctrl,
)

ctree.plot()
ctree.save()

In [ ]:
ypredict = ctree.predict(newx=test.drop(columns=["response"]), type="response")
ypredict_probs = ctree.predict(newx=test.drop(columns=["response"]), type="prob")

ctree.assess(ytrue=test["response"], ypredict=ypredict, ypredict_probs=ypredict_probs)